# Original Dataset

In [1]:
import pandas as pd
import yfinance as yf
import datetime
import numpy as np
import requests
from bcb import sgs
import ipeadatapy as ipea

# Defining start and end dates
start = datetime.datetime(2006, 1, 1)
end = datetime.datetime(2023, 12, 1)

# Import IBOV data
ibov = yf.download('^BVSP', start=start, end=end)


[*********************100%%**********************]  1 of 1 completed


In [2]:
# Visualizing the first rows of the IBOV data
print(ibov.head())

               Open     High      Low    Close  Adj Close  Volume
Date                                                             
2006-01-02  33462.0  33519.0  32860.0  33507.0    33507.0       0
2006-01-03  33507.0  34563.0  33507.0  34541.0    34541.0       0
2006-01-04  34540.0  35223.0  34540.0  35002.0    35002.0       0
2006-01-05  35006.0  35088.0  34681.0  34936.0    34936.0       0
2006-01-06  35170.0  35529.0  34940.0  35475.0    35475.0       0


In [3]:
# Dataframe Shape
tamanho = ibov.shape


print("Number of rows:", tamanho[0])
print("Number of columns:", tamanho[1])

Number of rows: 4431
Number of columns: 6


##  Basic

In [4]:
# Volume variation
ibov['Volume_Variation'] = ibov['Volume'].pct_change()
ibov['Volume_Variation'] = ibov['Volume_Variation'].replace([float('inf'), -float('inf')], pd.NA).fillna(0) * 100

# Price lags
for i in range(1, 6):
    ibov[f'Close_Lag_{i}'] = ibov['Close'].shift(i)
    ibov[f'Open_Lag_{i}'] = ibov['Open'].shift(i)
    ibov[f'High_Lag_{i}'] = ibov['High'].shift(i)
    ibov[f'Low_Lag_{i}'] = ibov['Low'].shift(i)


C:\Users\stutz\AppData\Local\Temp\ipykernel_1928\1279284508.py:3: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  ibov['Volume_Variation'] = ibov['Volume_Variation'].replace([float('inf'), -float('inf')], pd.NA).fillna(0) * 100


## Technical 

### Relative Strength Index

In [5]:
# Price change
ibov['Price_Change'] = ibov['Close'].diff()

# Gains and losses
gain = np.where(ibov['Price_Change'] > 0, ibov['Price_Change'], 0)
loss = np.where(ibov['Price_Change'] < 0, -ibov['Price_Change'], 0)

# Window side for future calculation
window_size = 14

# Exponential moving averages for gains and losses
avg_gain = pd.Series(gain, index=ibov.index).ewm(span=window_size, adjust=False).mean()
avg_loss = pd.Series(loss, index=ibov.index).ewm(span=window_size, adjust=False).mean()

# RSI
rs = avg_gain / avg_loss
rsi = 100 - (100 / (1 + rs))

# Add RSI to the dataframe
ibov['RSI'] = rsi

### Simple Moving Average 10 days

In [6]:
ibov['SMA_10'] = ibov['Close'].rolling(window=10).mean()

### Simple Moving Average 30 days

In [7]:
ibov['SMA_30'] = ibov['Close'].rolling(window=30).mean()

### Simple Moving Average 100 days

In [8]:
ibov['SMA_100'] = ibov['Close'].rolling(window=100).mean()

### On-Balance Volume

In [9]:
# Initialize the OBV with the first volume value
obv = [ibov['Volume'].iloc[0]]

# Iterate over the data, calculating the OBV
for i in range(1, len(ibov)):
    if ibov['Close'].iloc[i] > ibov['Close'].iloc[i - 1]:
        obv.append(obv[-1] + ibov['Volume'].iloc[i])
    elif ibov['Close'].iloc[i] == ibov['Close'].iloc[i - 1]:
        obv.append(obv[-1])
    else:
        obv.append(obv[-1] - ibov['Volume'].iloc[i])

# Add the OBV to the DataFrame
ibov['OBV'] = obv


### Exponential Moving Average 10 days

In [10]:
ibov['EMA_10'] = ibov['Close'].ewm(span=10, adjust=False).mean()

### Exponential Moving Average 30 days

In [11]:
ibov['EMA_30'] = ibov['Close'].ewm(span=30, adjust=False).mean()

### Exponential Moving Average 100 days

In [12]:
ibov['EMA_100'] = ibov['Close'].ewm(span=100, adjust=False).mean()

### Moving Average Convergenge/Divergence (MACD)

In [13]:
# Calculate the 12-period Exponential Moving Average (EMA)
ema_12 = ibov['Close'].ewm(span=12, adjust=False).mean()

# Calculate the 26-period Exponential Moving Average (EMA)
ema_26 = ibov['Close'].ewm(span=26, adjust=False).mean()

# Calculate the MACD
macd = ema_12 - ema_26

# Calculate the 9-period Exponential Moving Average (EMA) of the MACD to get the signal line
signal_line = macd.ewm(span=9, adjust=False).mean()

# Add MACD and signal line to the DataFrame
ibov['MACD'] = macd
ibov['MACD_Signal'] = signal_line

### Bollinger Bands

In [14]:
N = 20 # Moving Average periods 
k = 2  # Factor to shift the bands

ibov['CP_Standard_Deviation'] = ibov['Close'].rolling(N).std()
ibov['Middle_Band'] = ibov['Close'].rolling(N).mean()
ibov['Upper_Band'] = ibov['Middle_Band'] + ibov['CP_Standard_Deviation'] * k
ibov['Lower_Band'] = ibov['Middle_Band'] - ibov['CP_Standard_Deviation'] * k

### Money Flow Index

In [15]:
# Calculate the typical price
ibov['Typical_Price'] = (ibov['Close'] + ibov['High'] + ibov['Low']) / 3

# Calculate the Money Flow
ibov['Money_Flow'] = ibov['Typical_Price'] * ibov['Volume']

# Define the period for the MFI calculation
period = 14

# Calculate positive and negative flows
positive_flow = ibov['Money_Flow'].where(ibov['Typical_Price'].diff() > 0, 0)
negative_flow = ibov['Money_Flow'].where(ibov['Typical_Price'].diff() < 0, 0)

# Calculate the accumulated money flows for the defined period
positive_mf = positive_flow.rolling(window=period).sum()
negative_mf = negative_flow.rolling(window=period).sum()

# Calculate the Money Flow Index (MFI)
mfi = 100 * (positive_mf / (positive_mf + negative_mf))

# Add the MFI to the DataFrame
ibov['MFI'] = mfi

# Remove intermediate columns
ibov.drop(['Typical_Price', 'Money_Flow'], axis=1, inplace=True)

### William's %R

In [16]:
period = 14
high = ibov['High'].rolling(window=period).max()
low = ibov['Low'].rolling(window=period).min()
ibov['Williams_%R'] = (high - ibov['Close']) / (high - low) * -100


## Overseas

### Indices

In [17]:
# List of the main stock market indices
indices = {
    'SP500': '^GSPC',      # S&P 500 (USA)
    'DJIA': '^DJI',        # Dow Jones Industrial Average (USA)
    'NASDAQ': '^IXIC',     # NASDAQ Composite (USA)
    'NYSE': '^NYA',        # NYSE Composite (USA)
    'FTSE100': '^FTSE',    # FTSE 100 (United Kingdom)
    'DAX': '^GDAXI',       # DAX (Germany)
    'CAC40': '^FCHI',      # CAC 40 (France)
    'NIKKEI': '^N225',     # Nikkei 225 (Japan)
    'HSI': '^HSI',         # Hang Seng Index (Hong Kong)
    'ASX200': '^AXJO',     # ASX 200 (Australia)
    'KOSPI': '^KS11',      # KOSPI (South Korea)
    'TSEC': '^TWII',       # Taiwan Capitalization Weighted Stock Index (Taiwan)
    'SSECI': '000001.SS',  # SSE Composite Index (China)
    'Shenzhen': '399001.SZ',
    'Vix': '^VIX'
}

# Download historical data for all indices
data = {}
for index, ticker in indices.items():
    data[index] = yf.download(ticker, start=start, end=end)['Close']

# Create a DataFrame with the data
indices_df = pd.DataFrame(data)

# Add the indices data to the main DataFrame
ibov = ibov.join(indices_df, how='left')


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

## Macro

### Câmbio

In [18]:
# Dictionary of currency pairs in relation to the Real (BRL) on Yahoo Finance
currencies_yf = {
    'USD/BRL': 'USDBRL=X',  # US Dollar
    'GBP/BRL': 'GBPBRL=X',  # British Pound
    'EUR/BRL': 'EURBRL=X'   # Euro
}

# Download historical data for all currency pairs from Yahoo Finance
data_currencies_yf = {}
for currency, ticker in currencies_yf.items():
    data_currencies_yf[currency] = yf.download(ticker, start=start, end=end)['Close']

# Create a DataFrame with the currency data from Yahoo Finance
currencies_yf_df = pd.DataFrame(data_currencies_yf)

# Add the currency data from Yahoo Finance to the main DataFrame
ibov = ibov.join(currencies_yf_df, how='left', rsuffix='_yf')


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


### Commodities

In [19]:
# Dictionary of commodity ticker codes
commodities = {
    'Gold': 'GC=F',      # Gold
    'Silver': 'SI=F',    # Silver
    'Crude_Oil': 'CL=F', # Crude Oil
    'Copper': 'HG=F',    # Copper
    'Natural_Gas': 'NG=F', # Natural Gas
    'Corn': 'ZC=F',      # Corn
    'Soy': 'ZS=F',       # Soy
    'Wheat': 'KE=F',     # Wheat
    'Live_Cattle': 'LE=F', # Live Cattle
    'Coffee': 'KC=F'     # Coffee
}

# Download historical data for all commodities
data_commodities = {}
for commodity, ticker in commodities.items():
    data_commodities[commodity] = yf.download(ticker, start=start, end=end)['Close']

# Create a DataFrame with the commodity data
commodities_df = pd.DataFrame(data_commodities)

# Add the commodity data to the main DataFrame (ibov)
ibov = ibov.join(commodities_df, how='left')

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


### Interest rates, bond yield, and monetary indicators

In [20]:
# Codes for time series from the Central Bank of Brazil
series_codes_daily = {
    'Selic_Target': 432,
    'Selic_Daily': 11,
    'CDI': 12,
    'IMA-S': 12462,
    'IRF-M': 12461,
    'IMA-B': 12466,
    'Currency_in_circulation': 1780,
    'Bank_Reserves': 1781,
    'Restricted_monetary_base': 1782
}

def get_bcb_data(series_code, start, end):
    response = sgs.get({'data': series_code}, start=start, end=end)
    return response.get('data', pd.DataFrame())  # Return empty DataFrame if 'data' is not in response

# Get additional daily data
data_daily = {name: get_bcb_data(code, start, end) for name, code in series_codes_daily.items()}

# Filter to ensure only valid data is used
data_daily_filtered = {k: v for k, v in data_daily.items() if not v.empty}

# Combine the additional daily data into a single DataFrame
if data_daily_filtered:
    df_daily = pd.DataFrame(data_daily_filtered)

    # Convert indices to datetime
    df_daily.index = pd.to_datetime(df_daily.index)

    # Combine data with the Ibov DataFrame
    ibov = ibov.join(df_daily, how='left')
else:
    print("No valid data was returned from the BCB time series.")

### EMBI+

In [21]:
embi = ipea.timeseries("JPM366_EMBI366", yearGreaterThan=2004, yearSmallerThan=2024)

# Combine the additional daily data into a single DataFrame
embi = pd.DataFrame(embi)

# Convert indices to datetime
embi.index = pd.to_datetime(embi.index)

# Join the 'ibov' and 'embi' DataFrames
ibov = ibov.join(embi[['VALUE (-)']], how='left')
ibov = ibov.rename(columns={'VALUE (-)': 'EMBI+'})

## Returns

In [22]:
# log returns
ibov['Log_Returns'] = np.log(ibov['Close'] / ibov['Close'].shift(1))

## Value at Risk

In [23]:
def calculate_historical_VaR(returns, window_size, alpha):
    VaR_series = returns.rolling(window=window_size).apply(lambda x: np.percentile(x, alpha * 100), raw=True)
    return VaR_series

# Calculate 10-day rolling VaR at 1% significance level
ibov['VaR_10day'] = calculate_historical_VaR(ibov['Log_Returns'], window_size=10, alpha=0.01)
ibov.tail(10)

,Open,High,Low,Close,Adj Close,Volume,Volume_Variation,Close_Lag_1,Open_Lag_1,High_Lag_1,...,CDI,IMA-S,IRF-M,IMA-B,Currency_in_circulation,Bank_Reserves,Restricted_monetary_base,EMBI+,Log_Returns,VaR_10day
Date,,,,,,,,,,,,,,,,,,,,,
2023-11-17,124639.0,125431.0,124547.0,125062.0,125062.0,15719100,-14.363464,124576.0,123165.0,124737.0,...,0.045513,NaN,NaN,NaN,327337271.0,81446709.0,408783980.0,203.0,0.003894,-0.002091
2023-11-20,124773.0,126162.0,124773.0,126095.0,126095.0,11108500,-29.331196,125062.0,124639.0,125431.0,...,0.045513,NaN,NaN,NaN,327260895.0,75129558.0,402390453.0,200.0,0.008226,-0.002091
2023-11-21,125957.0,125957.0,125060.0,125607.0,125607.0,10793500,-2.835666,126095.0,124773.0,126162.0,...,0.045513,NaN,NaN,NaN,327192546.0,82582596.0,409775143.0,199.0,-0.003878,-0.003723
2023-11-22,125626.0,126875.0,125439.0,126035.0,126035.0,14058500,30.249687,125607.0,125957.0,125957.0,...,0.045513,NaN,NaN,NaN,327128775.0,80322412.0,407451187.0,197.0,0.003402,-0.003723
2023-11-23,126035.0,126760.0,125764.0,126576.0,126576.0,7816400,-44.400896,126035.0,125626.0,126875.0,...,0.045513,NaN,NaN,NaN,327175126.0,80777856.0,407952982.0,NaN,0.004283,-0.003723
2023-11-24,126553.0,126553.0,125341.0,125542.0,125542.0,9613100,22.986285,126576.0,126035.0,126760.0,...,0.045513,NaN,NaN,NaN,327235630.0,81964276.0,409199906.0,196.0,-0.008203,-0.007813
2023-11-27,125517.0,125826.0,124840.0,125683.0,125683.0,11013800,14.570742,125542.0,126553.0,126553.0,...,0.045513,NaN,NaN,NaN,327551116.0,83445419.0,410996535.0,199.0,0.001122,-0.007813
2023-11-28,125726.0,126916.0,125388.0,126538.0,126538.0,12394200,12.533367,125683.0,125517.0,125826.0,...,0.045513,NaN,NaN,NaN,328069965.0,77620699.0,405690664.0,198.0,0.006780,-0.007813
2023-11-29,126541.0,127388.0,126018.0,126101.0,126101.0,12417500,0.187991,126538.0,125726.0,126916.0,...,0.045513,NaN,NaN,NaN,328778467.0,81958197.0,410736664.0,196.0,-0.003459,-0.007813


## Conditional Value at Risk

In [24]:
# Calculate CVaR based on the historical VaR
def calculate_historical_CVaR(returns, VaR_series, window_size, alpha):
    def cvar_function(window):
        VaR = np.percentile(window, alpha * 100)
        return window[window <= VaR].mean()

    CVaR_series = returns.rolling(window=window_size).apply(cvar_function, raw=True)
    return CVaR_series


# Calculate 10-day rolling CVaR at 1% significance level
ibov['CVaR_10day'] = calculate_historical_CVaR(ibov['Log_Returns'], ibov['VaR_10day'], window_size=10, alpha=0.01)
ibov.tail(10)

,Open,High,Low,Close,Adj Close,Volume,Volume_Variation,Close_Lag_1,Open_Lag_1,High_Lag_1,...,IMA-S,IRF-M,IMA-B,Currency_in_circulation,Bank_Reserves,Restricted_monetary_base,EMBI+,Log_Returns,VaR_10day,CVaR_10day
Date,,,,,,,,,,,,,,,,,,,,,
2023-11-17,124639.0,125431.0,124547.0,125062.0,125062.0,15719100,-14.363464,124576.0,123165.0,124737.0,...,NaN,NaN,NaN,327337271.0,81446709.0,408783980.0,203.0,0.003894,-0.002091,-0.002158
2023-11-20,124773.0,126162.0,124773.0,126095.0,126095.0,11108500,-29.331196,125062.0,124639.0,125431.0,...,NaN,NaN,NaN,327260895.0,75129558.0,402390453.0,200.0,0.008226,-0.002091,-0.002158
2023-11-21,125957.0,125957.0,125060.0,125607.0,125607.0,10793500,-2.835666,126095.0,124773.0,126162.0,...,NaN,NaN,NaN,327192546.0,82582596.0,409775143.0,199.0,-0.003878,-0.003723,-0.003878
2023-11-22,125626.0,126875.0,125439.0,126035.0,126035.0,14058500,30.249687,125607.0,125957.0,125957.0,...,NaN,NaN,NaN,327128775.0,80322412.0,407451187.0,197.0,0.003402,-0.003723,-0.003878
2023-11-23,126035.0,126760.0,125764.0,126576.0,126576.0,7816400,-44.400896,126035.0,125626.0,126875.0,...,NaN,NaN,NaN,327175126.0,80777856.0,407952982.0,NaN,0.004283,-0.003723,-0.003878
2023-11-24,126553.0,126553.0,125341.0,125542.0,125542.0,9613100,22.986285,126576.0,126035.0,126760.0,...,NaN,NaN,NaN,327235630.0,81964276.0,409199906.0,196.0,-0.008203,-0.007813,-0.008203
2023-11-27,125517.0,125826.0,124840.0,125683.0,125683.0,11013800,14.570742,125542.0,126553.0,126553.0,...,NaN,NaN,NaN,327551116.0,83445419.0,410996535.0,199.0,0.001122,-0.007813,-0.008203
2023-11-28,125726.0,126916.0,125388.0,126538.0,126538.0,12394200,12.533367,125683.0,125517.0,125826.0,...,NaN,NaN,NaN,328069965.0,77620699.0,405690664.0,198.0,0.006780,-0.007813,-0.008203
2023-11-29,126541.0,127388.0,126018.0,126101.0,126101.0,12417500,0.187991,126538.0,125726.0,126916.0,...,NaN,NaN,NaN,328778467.0,81958197.0,410736664.0,196.0,-0.003459,-0.007813,-0.008203


## Standard Deviation of Returns

In [25]:
window_size = 10
ibov['Std_Dev_Returns_10day'] = ibov['Log_Returns'].rolling(window=window_size).std()

## Maximum Drawdown 

In [26]:
# Definition of the MDD_Rel function
def MDD_Rel(X):
    # Convert the input to a 2D numpy array
    returns_array = np.array(X, ndmin=2)
    
    # If the input is a 1D array with multiple elements, transpose it to a column vector
    if returns_array.shape[0] == 1 and returns_array.shape[1] > 1:
        returns_array = returns_array.T
    
    # Ensure the input array has the shape (T, 1)
    if returns_array.shape[0] > 1 and returns_array.shape[1] > 1:
        raise ValueError("The input returns must have a shape of (T, 1)")

    # Calculate the cumulative product of returns to get the Net Asset Value (NAV) series
    prices = 1 + np.insert(np.array(returns_array), 0, 0, axis=0)
    NAV = np.cumprod(prices, axis=0)

    # Initialize variables to track the maximum drawdown
    value = 0
    peak = -99999

    # Iterate over the NAV series to find the maximum drawdown
    for i in NAV:
        if i > peak:
            peak = i
        # Calculate the drawdown: (Trough Value - Peak Value) / Peak Value
        DD = (i - peak) / peak
        if DD < value:
            value = DD

    # Convert the final drawdown value to a scalar
    value = np.array(value).item()

    return value

# Function to calculate the rolling MDD
def calculate_rolling_MDD(returns, window_size):
    MDD_series = returns.rolling(window=window_size).apply(lambda x: MDD_Rel(x), raw=False)
    return MDD_series

window_size = 10

# Calculating the MDD with a rolling window
ibov['MDD_10day'] = calculate_rolling_MDD(ibov['Log_Returns'], window_size)

### All Columns

In [27]:
print(", ".join(ibov.columns.tolist()))

Open, High, Low, Close, Adj Close, Volume, Volume_Variation, Close_Lag_1, Open_Lag_1, High_Lag_1, Low_Lag_1, Close_Lag_2, Open_Lag_2, High_Lag_2, Low_Lag_2, Close_Lag_3, Open_Lag_3, High_Lag_3, Low_Lag_3, Close_Lag_4, Open_Lag_4, High_Lag_4, Low_Lag_4, Close_Lag_5, Open_Lag_5, High_Lag_5, Low_Lag_5, Price_Change, RSI, SMA_10, SMA_30, SMA_100, OBV, EMA_10, EMA_30, EMA_100, MACD, MACD_Signal, CP_Standard_Deviation, Middle_Band, Upper_Band, Lower_Band, MFI, Williams_%R, SP500, DJIA, NASDAQ, NYSE, FTSE100, DAX, CAC40, NIKKEI, HSI, ASX200, KOSPI, TSEC, SSECI, Shenzhen, Vix, USD/BRL, GBP/BRL, EUR/BRL, Gold, Silver, Crude_Oil, Copper, Natural_Gas, Corn, Soy, Wheat, Live_Cattle, Coffee, Selic_Target, Selic_Daily, CDI, IMA-S, IRF-M, IMA-B, Currency_in_circulation, Bank_Reserves, Restricted_monetary_base, EMBI+, Log_Returns, VaR_10day, CVaR_10day, Std_Dev_Returns_10day, MDD_10day


### Removing All NaN's

In [28]:
ibov = ibov.dropna()

### Converting to CSV

In [29]:
ibov.to_csv('ibovFinal.csv', index=True)